In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
ollama_api_key = "ollama"


gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

#Initialize clients for each provider using openai library

#anthropic = OpeanAi(api_key=anthropic_api_key,url=anthropic_url)
gemini = OpenAI(api_key=google_api_key,base_url=gemini_url)
openrouter = OpenAI(api_key=openrouter_api_key,base_url=openrouter_url)
ollama = OpenAI(api_key=ollama_api_key,base_url=ollama_url)

gemeni_model="gemini-2.5-flash"
xiomi_model = "xiaomi/mimo-v2-flash:free"
ollma_model = "llama2.3"


In [3]:
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant"

In [4]:
openrouter = OpenAI(api_key=openrouter_api_key,base_url=openrouter_url)

In [5]:
def chat(message, history):
    return "bananas"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [13]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [ ]:
gr.ChatInterface(fn=chat,type="messages").launch()

In [11]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openrouter.chat.completions.create(model=xiomi_model, messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [15]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = openrouter.chat.completions.create(model=xiomi_model, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [17]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [21]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    if 'car' in message.lower():
        relevant_system_message += "Offer a seat belt"
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openrouter.chat.completions.create(model=xiomi_model, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [23]:
import json

system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openrouter.chat.completions.create(model=xiomi_model, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [26]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [27]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $799'

In [28]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [29]:
tools = [{"type": "function", "function": price_function}]

In [30]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [31]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openrouter.chat.completions.create(model=xiomi_model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openrouter.chat.completions.create(model=xiomi_model, messages=messages)
    
    return response.choices[0].message.content

In [32]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()